In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
from Bio import Entrez
import xml.etree.ElementTree as ET
import os
import pickle

In [2]:
filename = 'non_pmc.txt'

with open(filename, 'r') as file:
    pmids_without_pmc = [line.strip() for line in file if line.strip()]

print(len(pmids_without_pmc))
print(pmids_without_pmc)

140080
['17684787', '4119740', '20109816', '12620731', '4128839', '16718861', '55713', '64838', '7783533', '15351201', '11558525', '7907763', '1359302', '13824259', '7491012', '1975328', '11567695', '12672326', '9014905', '46026', '80660', '2892007', '15302171', '4133468', '2563440', '8684209', '16198767', '30894256', '14346760', '14074486', '14258560', '2885530', '1681301', '16054934', '29031847', '13550970', '81971', '8105169', '6143895', '27979399', '65601', '1357500', '8093801', '82880', '6112491', '2874293', '8096589', '4112590', '10560692', '23746894', '2858729', '11081545', '13153114', '4123583', '9716058', '86894', '7837892', '7968068', '8103830', '47992', '14328808', '2571845', '10326580', '22405793', '12620758', '4189764', '7603143', '7637485', '11425422', '2899736', '24139110', '8538335', '26369473', '35526541', '31954472', '11830217', '4170902', '1348348', '2857316', '12057576', '13994893', '6170852', '4110653', '84239', '27411870', '4126962', '16458756', '34364519', '28569

In [10]:
with open("store_urls.pkl", 'rb') as f:
    full_text_url = pickle.load(f)
print(len(full_text_url))

1820


In [4]:
def fetch_doi(pm_id):
    Entrez.email = "ae22b024@smail.iitm.ac.in"
    try:
        handle = Entrez.efetch(db="pubmed", id=pm_id, rettype="full", retmode="xml")
        xml_data = handle.read()
        handle.close()
    except Exception as e:
        print(f"Error fetching data for PM ID {pm_id}: {e}")
        return False
    
    root = ET.fromstring(xml_data)

    def language_check():
        language_tag = root.find('.//Language')
        if language_tag is not None and language_tag.text is not None:
            if language_tag.text == "eng":
                return True
            else:
                return False
        return True    # might become a problem in future
    
    doi_found = None
    for eloc in root.findall('.//ELocationID'):
        if eloc.get('EIdType') == 'doi':
            doi_found = eloc.text.strip()
            break
    
    if doi_found is None:
        for article_id in root.findall('.//ArticleId'):
            if article_id.get('IdType') == 'doi':
                doi_found = article_id.text.strip()
                break
    
    if language_check:
        return doi_found
    else:
        return None

In [11]:
# Extracting link from the bottom of the page (Also can we conclude if there is link on the bottom of the page then it's there on the side also)
import requests
from bs4 import BeautifulSoup

def language_check_for_bottom_link(pm_id):
    Entrez.email = "ae22b024@smail.iitm.ac.in"
    try:
        handle = Entrez.efetch(db="pubmed", id=pm_id, rettype="full", retmode="xml")
        xml_data = handle.read()
        handle.close()
    except Exception as e:
        print(f"Error fetching data for PM ID {pm_id}: {e}")
        return False
    
    root = ET.fromstring(xml_data)

    language_tag = root.find('.//Language')
    if language_tag is not None and language_tag.text is not None:
        if language_tag.text == "eng":
            return True
        else:
            return False
    return True    # might become a problem in future


def extract_full_text_url(url, pm_id):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        div_element = soup.find('div', class_='full-text-links-list')

        if div_element != None:
          anchor_element = div_element.find('a', class_='link-item')
          href = anchor_element.get('href')
          if language_check_for_bottom_link(pm_id):
              #print(href, pm_id)
              return href
          else:
              #print("None", pm_id)
              return None
          
        else:
          print("None")
          x = fetch_doi(pm_id)
          if x is not None:
            url = f"https://doi.org/{x}"
            return url
          else:
            #print("both cases failed", pm_id)
            return None
          

    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        x = fetch_doi(pm_id)
        if x is not None:
            url = f"https://doi.org/{x}"
            return url
        else:
            #print("both cases failed", pm_id)
            return None
        
for i in range(1820, len(pmids_without_pmc)):
  full_text_url.append(extract_full_text_url('https://pubmed.ncbi.nlm.nih.gov/' + f'{pmids_without_pmc[i]}/', pmids_without_pmc[i]))
  if i % 100 == 0:
     print(i)

None
Error fetching data for PM ID 6147536: HTTP Error 500: Internal Server Error
Error fetching data for PM ID 28905739: HTTP Error 500: Internal Server Error
Error fetching data for PM ID 4101638: HTTP Error 500: Internal Server Error
None
Error fetching data for PM ID 8105338: HTTP Error 500: Internal Server Error
Error fetching data for PM ID 9314861: HTTP Error 500: Internal Server Error
None
None
None
None
None
1900
None
None
None
None
None
None
None
None
None
2000
None
None
None
None
None
None
None
None
None
None
None
None
None
None
2100
None
None
None
None
None
None
None
None
None
None
None
Error fetching data for PM ID 4106105: Remote end closed connection without response
None
None
2200
None
None
None
Failed to retrieve the webpage. Status code: 502
None
None
None
None
None
None
None
None
None
None
None
None
None
2300
None
None
None
None
None
None
None
None
None
None
2400
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
2500
None
None
None
None
None


KeyboardInterrupt: 

In [6]:
print(len(full_text_url))
count = 0
for i in range(len(full_text_url)):
    if full_text_url[i] == None:
        count += 1
print(count)

5
4


In [34]:
import pickle
with open("store_urls.pkl", 'wb') as f:
    pickle.dump(full_text_url, f)

In [31]:
'''def get_full_text_table(full_url, j):
    driver = webdriver.Chrome()
    driver.get(full_url)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    p_tags = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
    text = ' '.join([p.get_text() for p in p_tags])

    tables = soup.find_all('table')
    for table in tables:
        headers = [th.text.strip() for th in table.find_all('th')]
        rows = []
        for tr in table.find_all('tr'):
            cells = tr.find_all('td')
            if not cells:
                continue
            row = [cell.text.strip() for cell in cells]
            rows.append(row)

        df = pd.DataFrame(rows, columns=headers)
        print(df)
        print("Table saved.")

    print("Text extracted.")
    #print(text)
    with open(f'extracted_text_{j}.txt', 'w', encoding='utf-8') as file:
        #file.write("\n\n\n\n")
        file.write(text)

    driver.quit()

for j, url in enumerate(full_text_url, start=1):
    if url:
        get_full_text_table(url, j)
    else:
        print("can't extract: maybe not written in english")'''

'def get_full_text_table(full_url, j):\n    driver = webdriver.Chrome()\n    driver.get(full_url)\n\n    html = driver.page_source\n    soup = BeautifulSoup(html, \'html.parser\')\n\n    p_tags = soup.find_all([\'p\', \'h1\', \'h2\', \'h3\', \'h4\', \'h5\', \'h6\'])\n    text = \' \'.join([p.get_text() for p in p_tags])\n\n    tables = soup.find_all(\'table\')\n    for table in tables:\n        headers = [th.text.strip() for th in table.find_all(\'th\')]\n        rows = []\n        for tr in table.find_all(\'tr\'):\n            cells = tr.find_all(\'td\')\n            if not cells:\n                continue\n            row = [cell.text.strip() for cell in cells]\n            rows.append(row)\n\n        df = pd.DataFrame(rows, columns=headers)\n        print(df)\n        print("Table saved.")\n\n    print("Text extracted.")\n    #print(text)\n    with open(f\'extracted_text_{j}.txt\', \'w\', encoding=\'utf-8\') as file:\n        #file.write("\n\n\n\n")\n        file.write(text)\n\n    

In [23]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup

def get_full_text_table(full_url, j):
    driver = webdriver.Chrome()
    try:
        driver.get(full_url)
        time.sleep(4)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        # Find the article tag
        article = soup.find('article')

        # Find all p and header (h1 to h6) tags inside the article tag
        if article:
            elements = article.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

            # Collect the text content of each element in the correct order
            full_text = [element.text.strip() for element in elements]
            full_text = "\n".join(full_text)

            # Join the collected text into a single string with newlines
            if full_text.strip():
                
                print("Text extracted.")
                with open(f'C:\\Users\\Ajay Kanna\\Desktop\\UGRC\\extract abstract\\Lancet\\full_text_stored\\extracted_text_{j}.txt', 'w', encoding='utf-8') as file:
                    file.write(full_text)
        else:
            print("No article tag found")
    finally:
        driver.quit()

for j, url in enumerate(full_text_url, start=1):
    if url:
        get_full_text_table(url, j)
    else:
        print("can't extract: maybe, not written in English")

Text extracted.
Text extracted.
Text extracted.
Text extracted.
Text extracted.
Text extracted.


In [13]:
print(full_text_url)

['https://linkinghub.elsevier.com/retrieve/pii/S0140-6736(24)00968-1', 'https://linkinghub.elsevier.com/retrieve/pii/S0140-6736(24)00596-8', 'https://linkinghub.elsevier.com/retrieve/pii/S0140-6736(14)62245-5', 'https://linkinghub.elsevier.com/retrieve/pii/S0140-6736(17)32465-0', 'https://linkinghub.elsevier.com/retrieve/pii/S0140-6736(17)32752-6', 'https://doi.org/10.1016/s0140-6736(75)91102-2']
